In [1]:
import re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import plotly.express as px

def get_soup(input_url,parser,find):
    url = input_url
    r = requests.get(url)
    if find == "content" :
        soup = BeautifulSoup(r.content, parser)
    else :
        soup = BeautifulSoup(r.text, parser)
    return soup

In [2]:
pandas_wiki_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_Supreme_Court_justices_by_time_in_office')[1]
# date_errors = pandas_wiki_df[pd.to_datetime(pandas_wiki_df[pandas_wiki_df.columns[4]], errors="coerce").isna()]
# date_errors
# pandas_wiki_df[pd.to_datetime(pandas_wiki_df[pandas_wiki_df.columns[4]], errors="coerce").isna()==False]

In [3]:
table = get_soup('https://en.wikipedia.org/wiki/List_of_United_States_Supreme_Court_justices_by_time_in_office','html.parser','content').find( "table", {"id":"justices"} )
justices_tr = table.findAll("tr")[1:]

justice_name = []
justice_url = []
justice_term_start = []
justice_term_end = []

for justice in justices_tr:
    justice_str = justice.find_all('td')[1].text
    justice_name.append(' '.join(justice_str.split()))

    justiceurl = justice.find('a').get('href')
    justice_url.append(justiceurl)
    
    # justice_ts = justice.find_all('td')
    # justice_term_start.append(justice_ts[4].text)

    # justice_te = justice.find_all('td')
    # justice_term_end.append(justice_te[5].text)

justice_wiki_link_id = pd.DataFrame({
                                    'name': justice_name,
                                    'url': justice_url,
                    # 'term_start': justice_term_start,
                    # 'term_end':justice_term_end
                    })
justice_wiki_link_id 

,name,url
0,William O. Douglas,/wiki/William_O._Douglas
1,Stephen Johnson Field,/wiki/Stephen_Johnson_Field
2,John Paul Stevens,/wiki/John_Paul_Stevens
3,John Marshall (CJ),/wiki/John_Marshall
4,Hugo Black,/wiki/Hugo_Black
...,...,...
110,Robert Trimble,/wiki/Robert_Trimble
111,Amy Coney Barrett,/wiki/Amy_Coney_Barrett
112,John Rutledge (CJ),/wiki/John_Rutledge
113,Thomas Johnson,/wiki/Thomas_Johnson_(jurist)


In [4]:
urls = []
b_date = []
d_date = []

def parseWikiDateData(wiki_date_data_str,y,m,d):
    try:
        year = wiki_date_data_str[y].replace(r"}}", "")
        month = wiki_date_data_str[m].replace(r"}}", "")
        day = wiki_date_data_str[d].replace(r"}}", "")
        date = year+"-"+month+"-"+day
    except:
        date = year+"-"+"01"+"-"+"01"
    return pd.to_datetime(date)

def get_birth_death(justice_url):
    try:
        format_justice_url = 'https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&rvsection=0&titles='+justice_url.replace("/wiki/", "")+'&format=xml'
        soup = get_soup(format_justice_url,'xml','text')
        urls.append(justice_url)
        # print(justice_url.replace("/wiki/", ""))

        birth_re = re.search(r'(birth_date(.*?)}})', soup.getText())
        birth_data = birth_re.group(0).split('|')

        birth_date = parseWikiDateData(birth_data,1,2,3)
        b_date.append(birth_date)

        try:
            death_re = re.search(r'(death_date(.*?)}})', soup.getText())
            death_data = death_re.group(0).split('|')
            # print(death_data)
            if death_data[1] == r'{{death date and age':
                death_date = parseWikiDateData(death_data,2,3,4)
                d_date.append(death_date)
            elif death_data[0] > death_data[4]:
                death_date = parseWikiDateData(death_data,1,2,3)
                d_date.append(death_date)
            else:
                death_date = parseWikiDateData(death_data,4,5,6)
                d_date.append(death_date)
        except:
            d_date.append(np.nan)

    except:
        print(justice_url.replace("/wiki/", ""))
        b_date.append(np.nan)
        d_date.append(np.nan)
        

for url in justice_wiki_link_id['url']:
    get_birth_death(url)





James_Moore_Wayne
David_Josiah_Brewer
James_Wilson_(justice)
Wiley_Blount_Rutledge


In [5]:
birth_death_table = pd.DataFrame({
                                'urls': urls,
                                'b_date':b_date,
                                'd_date':d_date,
                                })
workflow_combine_df = pandas_wiki_df.merge(justice_wiki_link_id ,how="left",left_on="Justice",right_on="name").merge(birth_death_table,how="left",left_on="url",right_on="urls")
formatted_df = pd.concat([workflow_combine_df[pd.to_datetime(workflow_combine_df[workflow_combine_df.columns[4]], 
                                            errors="coerce").isna()==False]])
# formatted_df


In [6]:
formatted_df['start_date'] = pd.to_datetime(formatted_df[formatted_df.columns[4]], 
                                            errors="coerce")
formatted_df['end_date'] = pd.to_datetime(np.where(formatted_df[formatted_df.columns[5]]=="Incumbent",
                                    pd.to_datetime("today").strftime("%Y-%m-%d"),
                                    formatted_df[formatted_df.columns[5]]
                                    ))
formatted_df['d_date'] = np.where(formatted_df['end_date']==pd.to_datetime("today").strftime("%Y-%m-%d"),
                                    formatted_df['end_date'],
                                    formatted_df['d_date'])    
formatted_df = formatted_df.sort_values(by="end_date", ascending=False)
plotly_df = formatted_df[['Justice','start_date','end_date','b_date','d_date']]
life = plotly_df[['Justice','b_date','d_date']].copy().rename(columns={'b_date':'start_date','d_date':'end_date'})
life['task'] = 'life'
term = plotly_df[['Justice','start_date','end_date']].copy()
term['task'] = 'term'
df = pd.concat([life,term]).dropna().sort_values(by=["end_date","task"], ascending=False)
# df[df['Justice']=="Antonin Scalia"]
# df[df['start_date']>="1900-01-01"]
# df

1894-07-20 00:00:00


In [8]:
# workflow_combine_df[workflow_combine_df['d_date'].isnull()]

In [9]:
lookup_df = life
lookup = lookup_df[lookup_df.end_date.isnull()]['Justice'].unique()
workflow_combine_df[workflow_combine_df['Justice'].isin(lookup)]

,Rank,Justice,Lengthin days,Lengthin years and days,Start date[3],End date[3],name,url,urls,b_date,d_date
9,10,James Moore Wayne,11860,"32 years, 172 days","January 14, 1835","July 5, 1867",James Moore Wayne,/wiki/James_Moore_Wayne,/wiki/James_Moore_Wayne,NaT,NaT
14,15,Anthony Kennedy,11121,"30 years, 163 days","February 18, 1988","July 31, 2018",Anthony Kennedy,/wiki/Anthony_Kennedy,/wiki/Anthony_Kennedy,1936-07-23,NaT
28,29,Sandra Day O'Connor,8894,"24 years, 128 days","September 25, 1981","January 31, 2006",Sandra Day O'Connor,/wiki/Sandra_Day_O%27Connor,/wiki/Sandra_Day_O%27Connor,1930-03-26,NaT
43,44,David Josiah Brewer,7385,"20 years, 81 days","January 6, 1890","March 28, 1910",David Josiah Brewer,/wiki/David_Josiah_Brewer,/wiki/David_Josiah_Brewer,NaT,NaT
48,49,David Souter,6838,"18 years, 263 days","October 9, 1990","June 29, 2009",David Souter,/wiki/David_Souter,/wiki/David_Souter,1939-09-17,NaT
81,82,James Wilson,3242,"8 years, 320 days","October 5, 1789","August 21, 1798",James Wilson,/wiki/James_Wilson_(justice),/wiki/James_Wilson_(justice),NaT,NaT
89,90,Wiley Blount Rutledge,2399,"6 years, 207 days","February 15, 1943","September 10, 1949",Wiley Blount Rutledge,/wiki/Wiley_Blount_Rutledge,/wiki/Wiley_Blount_Rutledge,NaT,NaT
99,99 (tie),Charles Evans Whittaker,1832,"5 years, 6 days","March 25, 1957","March 31, 1962",NaN,NaN,NaN,NaT,NaT


In [10]:
colors = {}
colors['life'] =  'rgb(255, 0, 0)' #specify the color for the 'planned' schedule bars
colors['term'] = 'rgb(0, 0, 255)'  #specify the color for the 'actual' schedule bars

fig = px.timeline(
    df, 
    x_start="start_date", 
    x_end="end_date", 
    y="Justice",
    color='task',
    color_discrete_map = colors,
    # hover_name="Task Description"
    width=1200, height=3200
    )

# fig.update_yaxes(autorange="reversed")          #if not specified as 'reversed', the tasks will be listed from bottom up       
fig.data[1].width=0.35 # update the width of the 'Actual' schedule bars (the second trace of the figure)
fig.show()